# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_path = "../output_data/cities10.csv"
cities_pd = pd.read_csv(cities_path)

cities_pd


,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,cayenne,4.9333,-52.3333,73.40,94,20,3.44,GF,1619582009
1,1,rikitea,-23.1203,-134.9692,76.73,74,100,11.65,PF,1619581917
2,2,bredasdorp,-34.5322,20.0403,46.40,93,71,3.44,ZA,1619582135
3,3,enid,36.3956,-97.8784,64.40,82,20,8.05,US,1619582135
4,4,olinda,-8.0089,-34.8553,75.20,88,75,4.61,BR,1619581724
...,...,...,...,...,...,...,...,...,...,...
552,552,huainan,32.6264,116.9969,74.64,28,93,9.89,CN,1619582331
553,553,casa nova,-9.4167,-41.1333,78.80,69,40,5.75,BR,1619582331
554,554,hakui,36.8833,136.7833,64.40,59,75,13.80,JP,1619582332
555,555,fomboni,-12.2800,43.7425,81.95,75,100,4.79,KM,1619582332


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_pd[["Lat", "Lng"]].astype(float)
print (locations)
humidity_rate = cities_pd["Humidity"].astype(float)

         Lat       Lng
0     4.9333  -52.3333
1   -23.1203 -134.9692
2   -34.5322   20.0403
3    36.3956  -97.8784
4    -8.0089  -34.8553
..       ...       ...
552  32.6264  116.9969
553  -9.4167  -41.1333
554  36.8833  136.7833
555 -12.2800   43.7425
556  66.0000  111.5000

[557 rows x 2 columns]


In [4]:
#fig = gmaps.figure()

figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
good_cities_df = cities_pd.loc[cities_pd['Max_Temp'].between(64, 76)]
good_cities_df = good_cities_df.loc[cities_pd['Wind_Speed']<10]
good_cities_df = good_cities_df.loc[cities_pd['Cloudiness']==0]
good_cities_df

,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
115,115,aswan,24.0934,32.9070,73.29,25,0,9.28,EG,1619581983
140,140,vila velha,-20.3297,-40.2925,73.40,94,0,5.55,BR,1619582180
150,150,chokwe,-24.5333,32.9833,66.25,67,0,2.66,MZ,1619582183
172,172,oussouye,12.4850,-16.5469,73.40,88,0,6.91,SN,1619582192
228,228,deniliquin,-35.5333,144.9667,73.00,43,0,2.75,AU,1619582212
276,276,bardiyah,31.7561,25.0865,68.99,32,0,9.66,LY,1619582228
376,376,aksu,41.1231,80.2644,71.38,14,0,0.83,CN,1619582263
444,444,conceicao do araguaia,-8.2578,-49.2647,75.20,100,0,4.61,BR,1619582289
445,445,xai-xai,-25.0519,33.6442,66.69,80,0,3.18,MZ,1619582290
447,447,saint-joseph,-21.3667,55.6167,75.99,78,0,6.91,RE,1619582290


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_list = []
lat_df = []
lng_df = []
ind = 0

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
target_type = "lodging"

record_number = 1
set_number = 1
for lat in good_cities_df.Lat:
    lat_df.append(lat)
for lng in good_cities_df.Lng:
    lng_df.append(lng)
    


In [7]:
while ind <  len(lat_df):
    target_coordinates = str(lat_df[ind]) + ", " + str(lng_df[ind])
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
        }
    response = requests.get(base_url, params=params)
    places_data = response.json()
    hotel_name = places_data['results'][0]['name']
    hotel_list.append(hotel_name)
    print (hotel_name)
    ind +=1
            

Nuba Nile
Quality Suites Vila Velha
Complexo Sonho Real
Auberge du Routard
Riviana Motel
بورتو بردية
Pudong Holiday Hotel
Tarumã Tropical Hotel
Complexo da Familia
"Plantation Bed and Breakfast


In [8]:
good_cities_df['Hotel_Name'] = hotel_list
good_cities_df

,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
115,115,aswan,24.0934,32.9070,73.29,25,0,9.28,EG,1619581983,Nuba Nile
140,140,vila velha,-20.3297,-40.2925,73.40,94,0,5.55,BR,1619582180,Quality Suites Vila Velha
150,150,chokwe,-24.5333,32.9833,66.25,67,0,2.66,MZ,1619582183,Complexo Sonho Real
172,172,oussouye,12.4850,-16.5469,73.40,88,0,6.91,SN,1619582192,Auberge du Routard
228,228,deniliquin,-35.5333,144.9667,73.00,43,0,2.75,AU,1619582212,Riviana Motel
276,276,bardiyah,31.7561,25.0865,68.99,32,0,9.66,LY,1619582228,بورتو بردية
376,376,aksu,41.1231,80.2644,71.38,14,0,0.83,CN,1619582263,Pudong Holiday Hotel
444,444,conceicao do araguaia,-8.2578,-49.2647,75.20,100,0,4.61,BR,1619582289,Tarumã Tropical Hotel
445,445,xai-xai,-25.0519,33.6442,66.69,80,0,3.18,MZ,1619582290,Complexo da Familia
447,447,saint-joseph,-21.3667,55.6167,75.99,78,0,6.91,RE,1619582290,"""Plantation Bed and Breakfast"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row


# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in good_cities_df.iterrows()]
locations = good_cities_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map

humidity_rate = good_cities_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)



# Display figure

fig



Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…